This notebook implements FOOOF model fitting for resting state data. PSD and model fitting settings are based on [Donoghue et al](https://www.nature.com/articles/s41593-020-00744-x). Right now, this is set up to run FOOOF on a subject by subject basis. TO DO: scale up across subjects. 

In [5]:
import sys
!{sys.executable} -m pip install neurodsp --quiet

You should consider upgrading via the '/Users/christinamaher/Desktop/particle_filter/pf/bin/python -m pip install --upgrade pip' command.


In [3]:
# load required packages 
%matplotlib inline

import os
from collections import Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm

from scipy.signal import periodogram

# MNE & associated code
import mne
from mne.preprocessing import ICA, read_ica
from mne.utils import _time_mask

from autoreject import AutoReject, read_auto_reject
from autoreject.autoreject import _apply_interp

# FOOOF, and custom helper & utility functions
from fooof import FOOOF, FOOOFGroup
from fooof.objs import average_fg
from fooof.plts import plot_spectrum
from fooof.utils import trim_spectrum
from fooof.data import FOOOFSettings
from fooof.analysis import get_band_peak_fm, get_band_peak_fg

# Import spectral power functions
from neurodsp.spectral import compute_spectrum_welch, rotate_powerlaw

# Import utilities for loading and plotting data
from neurodsp.utils import create_times
from neurodsp.utils.download import load_ndsp_data
#from spectral import plot_power_spectra
from neurodsp.plts.time_series import plot_time_series

ModuleNotFoundError: No module named 'fooof'

In [ ]:
# load data
subject_list = [] # list of subjects
sr = clean_raw.info[''] # get sampling rate 

In [ ]:
freq = [] # frequency - initialize list to store frequency for each channel
psd = [] # power spectral density - initialize list to store psd for each channel
num_channels = len(clean_raw.ch_names) # get number of channels
ch_names = clean_raw.ch_names # get channel names

for channel in range(num_channels):
    freq, psd = compute_spectrum_welch(sig=clean_raw._data[channel,:], fs=sr, window='hann',avg_type='median',nperseg=sr*2,f_range=(2,40))
    psd.append(psd)
    if channel == 0: # only need to get frequency once
        freq.append(freq)
    else:
        continue # continue to next channel

In [ ]:
# Run FOOOF on the power spectra for each channel in each subject and store the results
fg = FOOOFGroup(peak_width_limits=[1, 8], max_n_peaks=6, min_peak_amplitude=0.15, peak_threshold=2.0)
fg.fit(freq, psd, freq_range=[2, 40])

# Plot a summary of the results across the group
fg.plot()

# Save out PDF reports of the FOOOFGroup fits, same as with FOOOF
fg.save_report('FOOOFGroup_report')

In [ ]:
# Extract aperiodic parameters
aps = fg.get_params('aperiodic_params')
exps = fg.get_params('aperiodic_params', 'exponent')

# Extract peak parameters
peaks = fg.get_params('peak_params')
cfs = fg.get_params('peak_params', 'CF')

# Extract goodness-of-fit metrics
errors = fg.get_params('error')
r2s = fg.get_params('r_squared')

# Create a dataframe to store the results
df = pd.DataFrame({
    'ch_name': chan_names,
    'aperiodic_exponent': exps,
    'aperiodic_offset': aps[:, 0],
    'error': errors,
    'r^2': r2s,
    'power': peaks[:, 0],
    'center_freq': cfs,
    'bandwidth': peaks[:, 2]
})

# for each center_freq in the dataframe, create another column and store whether it falls with the delta, theta, alpha, beta, or gamma band
df['freq_band'] = pd.cut(df['center_freq'], [0, 4, 8, 12, 30, 40], labels=['delta', 'theta', 'alpha', 'beta', 'gamma'])

# for each ch_name and sub_id, create another column and store the sub_id and ch_name as a unique identifier (helpful for visualizing/stats)
df['uniqueID'] = df['ch_name'] + '_' + df['subject']

In [7]:
# Initialize dataframe to store fooof results with YBA labels 
df_YBA = pd.DataFrame()

for subject in subject_list:
    YBA = pd.read_csv(f'YBA_{subject}.csv')
    df_temp = df[df['sub_id'] == subject]
    df_temp['yba_label'] = df_temp['ch_name'].map(YBA.set_index('ch_name')['YBA'])
    df_YBA = df_YBA.append(df_temp)